# TODO

## Load Libraries

In [2]:
import pandas as pd
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate

## Initialize Chain

In [3]:
prompt = PromptTemplate(
    template="""Ви - експерт із перефразування тексту. Ваше завдання переписати текст так, щоб виглядало, що це написала людина {type_} роду. Не додавайте лишнього тексту та додаткових слів, можна змінювати тільки рід або відмінки слів.

Текст:```{text}```
Перефразований текст:""",
    input_variables=['type_', 'text'],
)

In [4]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42})

chain = prompt | llm

## Load data (Ukr CVs)

In [6]:
df = pd.read_csv('../data/uk_data_samples.csv')
df.head()

,item_id,candidate_id,job_id,CV,Job Description,Job Position,lang
0,e765137d-40e0-5ae0-aaf9-a966f76f3621_1d98054f-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"Комунікабельний, серйозний, старанний, швидко ...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk
1,e765137d-40e0-5ae0-aaf9-a966f76f3621_6addf108-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,6addf108-43e3-5677-b565-5117c619f084,"Комунікабельний, серйозний, старанний, швидко ...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk
2,e765137d-40e0-5ae0-aaf9-a966f76f3621_13dbe82c-...,e765137d-40e0-5ae0-aaf9-a966f76f3621,13dbe82c-a0fa-5538-82a6-64f8268ece38,"Комунікабельний, серйозний, старанний, швидко ...",Твої задачі:\r\n-створення та розробка дизайну...,2D Artist,uk
3,db4c55c9-e3e6-5d58-99ea-06697e4547e9_1d98054f-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,1d98054f-4adf-5db0-8e2d-4dab904dfe44,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",**💎Нашому супергерою ми обіцяємо:**\r\n— цікав...,2D Artist,uk
4,db4c55c9-e3e6-5d58-99ea-06697e4547e9_6addf108-...,db4c55c9-e3e6-5d58-99ea-06697e4547e9,6addf108-43e3-5677-b565-5117c619f084,"\n1. 2D animator в ""Storzy"" (12.2020-02.2022)\...",Розробник та видавець мобільних ігор Mystery T...,2D Artist,uk


In [7]:
cvs = list(df["CV"].unique())

In [16]:
import random 
random.seed(10)
sample = random.choices(cvs, k=2)
sample


['Курси. Сертифікати.\r\n\r\nЛінкбілдінг. Онлайн курс Ігоря Рудника. \r\nSEO-школа Collaborator.\r\nтравень 2023 р.\r\n\r\nОснови веб розробки. HTML і CSS\r\nUdemy\r\nтравень 2023 р.\r\n\r\nSpecialist Internet Marketing  \r\nАкадемія інтернет-маркетингу WEBPromoExperts м. Київ\r\nвересень-грудень 2022 р\r\n\r\nSEO Specialist \r\n ITEA GIZ IT Education Academy м. Київ\r\n жовтень-листопад 2022 р\r\n\r\nВ процесі навчання виконувала наступні завдання: \r\n- збір семантичного ядра та його кластеризація,  опрацювання структури сайту;\r\n- внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту);\r\n- формування контент-плану, розробка технічного завдання для написання контенту;\r\n- написання оптимізованого контенту (стаття для інформаційного ресурсу);\r\n- зовнішня оптимізація (аналіз зовнішніх лінок конкурентів, пошук джерел розміщення посилань).\nOctober 2012 - January 2020\r\nNAAS of Ukraine. Institute of Potato - Accountant-eco

In [22]:
for i, cv in enumerate(sample):
    print(f"Sample {i+1}")
    print(f"Original:\n{cv}\n")
    print(f'Rewritten female:\n{chain.invoke({"text": cv, "type_":"жіночого"}).content}\n')
    print(f'Rewritten male: \n{chain.invoke({"text": cv, "type_": "чоловічого"}).content}\n')
    print("\n\n")

Sample 1
Original:
Курси. Сертифікати.

Лінкбілдінг. Онлайн курс Ігоря Рудника. 
SEO-школа Collaborator.
травень 2023 р.

Основи веб розробки. HTML і CSS
Udemy
травень 2023 р.

Specialist Internet Marketing  
Академія інтернет-маркетингу WEBPromoExperts м. Київ
вересень-грудень 2022 р

SEO Specialist 
 ITEA GIZ IT Education Academy м. Київ
 жовтень-листопад 2022 р

В процесі навчання виконувала наступні завдання: 
- збір семантичного ядра та його кластеризація,  опрацювання структури сайту;
- внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту);
- формування контент-плану, розробка технічного завдання для написання контенту;
- написання оптимізованого контенту (стаття для інформаційного ресурсу);
- зовнішня оптимізація (аналіз зовнішніх лінок конкурентів, пошук джерел розміщення посилань).
October 2012 - January 2020
NAAS of Ukraine. Institute of Potato - Accountant-economist

July 2008 - December 2009
LLC Bautech-Ukraine - 

## Classical NLP pipeline for changing gender in CVs

In [1]:
%pip install stanza

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 970.4/970.4 kB 4.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.8/433.8 kB 7.2 MB/s eta 0:00:00a 0:00:01
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation

In [77]:
import stanza
import pymorphy3

# ignore logs
import logging
logging.getLogger('stanza').setLevel(logging.ERROR)

def changer(morph_token, gender, token):
    if morph_token.tag.POS == 'VERB':
        if morph_token.tag.gender == 'masc' and gender == 'femn':
            token = morph_token.inflect({'femn'}).word

        elif morph_token.tag.gender == 'femn' and gender == 'masc':
            token = morph_token.inflect({'masc'}).word

        else:
            token = morph_token.word
    return token

def gender_verb_changer(text: str, gender: str = 'masc') -> str:
    """
    Function for changing verbs from masculine to feminine and vice versa for Ukrainian language

    Args:
        text: Ukrainian language text
        gender: gender to which the verbs should be changed. Default: male

    Returns:
        text with changed verbs to masculine or feminine
    """

    nlp = stanza.Pipeline(lang='uk', processors='tokenize,mwt,pos,lemma,depparse')
    doc = nlp(text)
    morph = pymorphy3.MorphAnalyzer(lang='uk')
    changer_count = 0
    for sentence in doc.sentences:
        has_subject = any(word.deprel == 'nsubj'  for word in sentence.words)
        
        # find head for я with deprel to nsubj
        head_ids = [] 
        for word in sentence.words:
            if word.text.lower() == 'я' and word.deprel == 'nsubj':
                head_ids.append(word.head)
        
        if not has_subject or head_ids != []:
            for word in sentence.words:
                token = word.text
                morph_token = morph.parse(token)[0]
                if word.deprel == 'root' or word.id in head_ids:
                    token = changer(morph_token, gender, token)
                    if word.id == 1:
                        token = token.capitalize()
                    text = text[:word.start_char+changer_count] + token + text[word.end_char+changer_count:]
                    changer_count += len(token) - len(word.text)
    return text

In [78]:
# test with masculine verb
text = 'Сьогодні я пішов на роботу, я відпочивав, вона працювала та співала. Він працював. Компанія заробляла кошти. Пішли на роботу. Програмував цей диплом все життя.'
print(gender_verb_changer(text, 'masc'))
print(gender_verb_changer(text, 'femn'))


text = """
Курси. Сертифікати.

Лінкбілдінг. Онлайн курс Ігоря Рудника. 
SEO-школа Collaborator.
травень 2023 р.

Основи веб розробки. HTML і CSS
Udemy
травень 2023 р.

Specialist Internet Marketing  
Академія інтернет-маркетингу WEBPromoExperts м. Київ
вересень-грудень 2022 р

SEO Specialist 
ITEA GIZ IT Education Academy м. Київ
жовтень-листопад 2022 р

В процесі навчання виконувала наступні завдання: 
- збір семантичного ядра та його кластеризація,  опрацювання структури сайту;
- внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту);
- формування контент-плану, розробка технічного завдання для написання контенту;
- написання оптимізованого контенту (стаття для інформаційного ресурсу);
- зовнішня оптимізація (аналіз зовнішніх лінок конкурентів, пошук джерел розміщення посилань).
October 2012 - January 2020
NAAS of Ukraine. Institute of Potato - Accountant-economist

July 2008 - December 2009
LLC Bautech-Ukraine - Accountant

February 2006 - June 2008
Private enterprise "Yulga" - Accountant
Looking for a project where I can get practical skills and grow professionally. Under the supervision of a specialist. Type of employment full-time or part-time, remote.
"""
print("\n\n")
print(gender_verb_changer(text, 'masc'))
print()
print(gender_verb_changer(text, 'femn'))

Сьогодні я пішов на роботу, я відпочивав, вона працювала та співала. Він працював. Компанія заробляла кошти. Пішли на роботу. Програмував цей диплом все життя.
Сьогодні я пішла на роботу, я відпочивала, вона працювала та співала. Він працював. Компанія заробляла кошти. Пішли на роботу. Програмувала цей диплом все життя.




Курси. Сертифікати.

Лінкбілдінг. Онлайн курс Ігоря Рудника. 
Seo-школа Collaborator.
травень 2023 р.

Основи веб розробки. Html і CSS
Udemy
травень 2023 р.

Specialist Internet Marketing  
Академія інтернет-маркетингу WEBPromoExperts м. Київ
вересень-грудень 2022 р

Seo Specialist 
ITEA GIZ IT Education Academy м. Київ
жовтень-листопад 2022 р

В процесі навчання виконував наступні завдання: 
- збір семантичного ядра та його кластеризація,  опрацювання структури сайту;
- внутрішня оптимізація (технічний аудит сайту, робота з метатегами, дублями сторінок, швидкістю завантаження сайту);
- формування контент-плану, розробка технічного завдання для написання контенту;
-